In [3]:
# do new env 
import openai
import os
import time
import base64
import requests
from dotenv import load_dotenv
#An example for quiz4 with associated images in folder images4
visionGptEvaluation = 'visionGptEvaluation'
FILE_PATH = 'data/quiz4.xlsx' # Path to your Excel file
MAX_TOKENS = "4000"
image_folder='images4/'
rule_of_conduct = 'Describe the image and then use the that description and the following symptoms to choose the correct answer with out explanation. At the end of the response write Correct Choice:(ONLY the correct letter and no spaces).'
rule_of_conduct2= '''This quiz tests your ability to identify some other causes of cheilitis in 16 patients.
Sore lips are surprisingly common. Cheilitis (inflamed lips) may be acute, relapsing or chronic. The cause may be environmental, infection, or an inflammatory dermatosis.
Intermittent and irregular itching, soreness, swelling and blistering may be due to contact allergy; this tends to affect adjacent skin. A thorough history is required prior to patch testing to identify potential allergens. These include metals, fragrances and preservatives and may be found in lipstick, other cosmetics, musical instruments, toothpaste and mouthwashes, rubber balloons and food.
Chronic inflammatory skin diseases that affect both mucosa and cutaneous surfaces include discoid lupus erythematosus and lichen planus; these may first affect the lips.
For each of the sixteen cases, study the image(s) and then answer the questions.'''

load_dotenv()

API_URL = "https://api.openai.com/v1/chat/completions"
API_KEY=  os.getenv("API_KEY")


MAX_RETRIES = 5  # Max number of retries 
RETRY_DELAY = 10



In [2]:
import pandas as pd

#utils.py
def encode_image(image_path):
    """
    Encode an image to base64 format.

    Args:
        image_path: Path to the image file.

    Returns:
        A base64-encoded representation of the image.
    """
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode("utf-8")
    
def excel_to_array(file_path, rule_of_conduct, image_folder):
    """
    Read an Excel file and convert it to a list of image-question pairs and correct answers.
    
    Args:
        file_path: The path to the Excel file.
        rule_of_conduct: A string with rules or instructions to prepend to each question.
        image_folder: The folder where images are stored.

    Returns:
        A tuple of pairs (list of dictionaries), correct_answers (list), and explanations (list).
    """
    df = pd.read_excel(file_path)
    df['question']=rule_of_conduct + df['question']
    df['image_path']=image_folder + df['image_path']
    correct_answers=[]
    pairs = df[['image_path', 'question']].to_dict(orient='records')
    correct_answers = df['Correct_Responses'].tolist()
    explanations = df['Explanation'].tolist()
    return pairs, correct_answers, explanations


# quiz_pairs, correct_answers= excel_to_array(file_path)

# print(quiz_pairs[0])
# print(correct_answers)

In [3]:
#api_handler
os.environ["OPENAI_API_KEY"]=API_KEY
newopenai = openai.api_key = os.environ.get('OPENAI_API_KEY')

def send_request(image_path, question):
    """
    Send request to OpenAI API with an image and a question.
    Args:
        image_path: Path to the image file.
        question: The question to ask the API.
        encode_image_function: Function to encode the image to base64.

    Returns:
        The API response as a JSON object.
    """
    headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {newopenai}"
    }
    result_array = []

    base64_image = encode_image(image_path)
    payload = {
        "model": "gpt-4-vision-preview",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": question
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": MAX_TOKENS
    }
    
    max_retries = MAX_RETRIES  # Max number of retries 
    retry_delay = RETRY_DELAY  # Initial delay in seconds (can be adjusted)

    for attempt in range(max_retries):
        response = requests.post(API_URL, headers=headers, json=payload)
        
        # Check if the response was successful
        if response.status_code == 200:
            return response.json()
        
        # If rate limit exceeded, handle the error
        elif response.status_code == 429:
            try:
                # Extract the wait time from the error message
                error_info = response.json()
                wait_time = float(error_info['error']['message'].split('Please try again in ')[1].split('s')[0])
                time.sleep(wait_time)
            except (IndexError, KeyError, ValueError):
                # If parsing the wait time fails, use a default delay
                time.sleep(retry_delay)
        else:
            # For other errors, raise an exception or handle as required
            response.raise_for_status()

    # If all retries fail, raise an exception or return a default error response
    raise Exception("Max retries reached for API request.")

def evaluate_responses(result_array, correct_answers, explanations):
    """
    Evaluate the responses against the correct answers.
    Args:
        result_array: Array of responses from the API.
        correct_answers: List of correct answers for each question.

    Returns:
        Tuple containing the score and the total number of questions.
    """
    actual_correct_choices = correct_answers  # Assuming this array has the same length as result_array
    score = 0
    correct_responses = []
    incorrect_responses = []

    for i in range(len(result_array)):
        user_choice = result_array[i]["ChoiceMade"]
        correct_choice = actual_correct_choices[i]

        if user_choice == correct_choice:
            score += 1  # Increment score for each correct answer
            correct_responses.append((i, user_choice))
        else:
            incorrect_responses.append((i, user_choice, correct_choice))
            

    # Calculate the percentage score
    total_questions = len(result_array)
    percentage_score = score / total_questions * 100

    # Print the results
    print(f"Your score is {percentage_score}%")

    if correct_responses:
            print("\nCorrect Responses:")
            for index, choice in correct_responses:
                print(f"Question {index + 1}: Your choice '{choice}' was correct.")

    if incorrect_responses:
            print("\nIncorrect Responses:")
            for index, user_choice, correct_choice in incorrect_responses:
                print(f"Question {index + 1}: Your choice '{user_choice}' was incorrect. Correct answer: '{correct_choice}'")
                print(f"Question {index + 1}:"  f" Instead for question {index + 1}:"+ "the correct reasoning is : "+ explanations[index])


In [4]:
def main_execution_function():
    """
    Main execution function for the quiz package.
    This function will orchestrate the reading of the Excel file,
    sending requests to the OpenAI API, and evaluating responses.
    """
    try:

        # Read the Excel file and get pairs of questions and answers
        quiz_pairs, correct_answers, explanations = excel_to_array(FILE_PATH, rule_of_conduct,image_folder)

        result_array = []
        print(quiz_pairs)
        # Iterate over the image-question pairs and send them to the API
        for pair in quiz_pairs:
            response = send_request(pair["image_path"], pair["question"])
            print(response)
            
            total_tokens = response['usage']['total_tokens']
            content = response['choices'][0]['message']['content']
                # Check if 'Correct Choice: ' is in the content
            if 'Correct Choice:' in content:
                # Split and strip any leading/trailing whitespace from the choice
                choice_made = content.split('Correct Choice:')[1].strip()
            else:
                choice_made = 'Not Found'  # Or any other placeholder indicating the choice was not found

            result_array.append({
                "Tokens": total_tokens,
                "response": content,
                "ChoiceMade": choice_made
            })

        evaluate_responses(result_array, correct_answers, explanations)
            
        
    except Exception as e:
        print(f"An error occurred: {e}")

# This condition ensures the script runs only when executed directly (not when imported as a module)
if __name__ == "__main__":
    main_execution_function()


[{'image_path': 'static/images4/quiz033.jpg', 'question': 'Describe the image and then use the that description and the following symptoms to choose the correct answer with out explanation. At the end of the response write Correct Choice:(ONLY the correct letter and no spaces).This man has cattle and he thinks he picked this up from them. Given the sharp boarders and his history, you would:\n\tA. \tTreat with miconazole cream 2% twice daily until clear\n\tB. \tTreat with clotrimazole cream 27 twice daily until clear\n\tC. \tGive Lamisil (terbinafine) 250 mg daily for one month\n\tD. \tPerform a scraping of the scale and KOH prep to insure diagnosis'}, {'image_path': 'static/images4/quiz034.jpg', 'question': 'Describe the image and then use the that description and the following symptoms to choose the correct answer with out explanation. At the end of the response write Correct Choice:(ONLY the correct letter and no spaces).This man has these lesions clustered around his wrists and ankl